In [0]:
########################################
# Instalacion de paquetes
if (!requireNamespace("caret"))
  install.packages("caret")
if (!requireNamespace("tidyverse"))
  install.packages("tidyverse")
if (!requireNamespace("ggpubr"))
  install.packages("ggpubr")
if (!requireNamespace("randomForest"))
  install.packages("randomForest")
if (!requireNamespace("doParallel"))
  install.packages("doParallel")
if (!requireNamespace("dplyr"))
  install.packages("dplyr")
if (!requireNamespace("kernlab"))
  install.packages("kernlab")
if (!requireNamespace("gbm"))
  install.packages("gbm")

In [0]:
##################################
# Include
library(caret)
library(tidyverse)
library(randomForest)
library(ggpubr)
library(doParallel)
library(dplyr)
library(kernlab)
library(gbm)

In [0]:
###################################
# Paralelismo
parallel::detectCores()

cl <- makePSOCKcluster(4)

registerDoParallel(cl)

In [0]:
####################################
# Lectura de los datos
datos_castillo <- read_csv("C:\\Users\\Sergio\\Desktop\\final_dataset_castillodelromeral.csv", 
    col_types = cols(.default = col_character()))
datos_castillo %>% mutate_if(is.character, as.factor)

In [0]:
#####################################
# Resumen de los datos
glimpse(datos_castillo)

In [0]:
####################################
# PREPROCESADO DE LOS DATOS
####################################
# Establecer NA de R
datos_castillo$NO[datos_castillo$NO == "N"] <- NA
datos_castillo$NO2[datos_castillo$NO2 == "N"] <- NA
datos_castillo$PM1.0[datos_castillo$PM1.0 == "N"] <- NA
datos_castillo$PM2.5[datos_castillo$PM2.5 == "N"] <- NA
datos_castillo$O3[datos_castillo$O3 == "N"] <- NA
datos_castillo$CO[datos_castillo$CO == "N"] <- NA
datos_castillo$NO2[datos_castillo$NO2 == "N"] <- NA

In [0]:
# Establecer todos los datos a numerico
datos_castillo$Hora <- as.numeric(datos_castillo$Hora)
datos_castillo$SO2 <- as.numeric(datos_castillo$SO2)
datos_castillo$NO <- as.numeric(datos_castillo$NO)
datos_castillo$PM1.0 <- as.numeric(datos_castillo$PM1.0)
datos_castillo$PM2.5 <- as.numeric(datos_castillo$PM2.5)
datos_castillo$O3 <- as.numeric(datos_castillo$O3)
datos_castillo$CO <- as.numeric(datos_castillo$CO)
datos_castillo$NO2 <- as.numeric(datos_castillo$NO2)

In [0]:
####################################
# Resumen de los datos
glimpse(datos_castillo)

# Obtener el numero de entradas
nrow(datos_castillo)

In [0]:
####################################
# Comprobar si hay alguna fila incompleta
any(!complete.cases(datos_castillo))

# Numero de datos ausentes por variable
map_dbl(datos_castillo, .f = function(x){sum(is.na(x))})

In [0]:
####################################
#Para eliminar las filas con valor nulo en la clase
datos_castillo <- datos_castillo[!is.na(datos_castillo$NO2),]
datos_castillo <- datos_castillo[!is.na(datos_castillo$NO),]
datos_castillo <- datos_castillo[!is.na(datos_castillo$PM1.0),]
datos_castillo <- datos_castillo[!is.na(datos_castillo$PM2.5),]
datos_castillo <- datos_castillo[!is.na(datos_castillo$O3),]
datos_castillo <- datos_castillo[!is.na(datos_castillo$CO),]
datos_castillo <- datos_castillo[!is.na(datos_castillo$SO2),]

In [0]:
####################################
# Numero de datos ausentes por variable
map_dbl(datos_castillo, .f = function(x){sum(is.na(x))})

# Obtener el numero de entradas
nrow(datos_castillo)

In [0]:
####################################
# Identificar que variables contienen valores ""
datos_castillo %>% map_lgl(.f = function(x){any(!is.na(x) & x == "")})

In [0]:
# Mirar la varianza de Class
datos_castillo %>% select(NO2) %>% nearZeroVar(saveMetrics = TRUE)

In [0]:
####################################
# Division de los datos en entrenamiento y testeo

set.seed(123)

In [0]:
# Se crean los indices de las observaciones de entrenamiento
train <- createDataPartition(y = datos_castillo$NO2, p = 0.8, list = FALSE, times = 1)
   
datos_train <- datos_castillo[train, ]
datos_test  <- datos_castillo[-train, ]

In [0]:
#####################################
#         RANDOM FOREST             #
#####################################

##################################
# 1.
# Metodo de entrenamiento repeatedcv sobre randomForest
#
# repeatedcv: permite hacer repeticiones
# trainControl: Controla los matices de calculo de la funcion train
# tuneLength: El parametro le dice al algoritmo que pruebe diferentes valores predeterminados
#   para el parametro principal
# number: numero de iteraciones de remuestreo
# metric: RMSE (Raiz del error cuadratico medio)
#
set.seed(123)
tc <- trainControl(method="repeatedcv", 
                   number=6, 
                   repeats=2, 
                   verboseIter = FALSE,
                   allowParallel = TRUE)
modelo <- train(NO2~.,
                data=datos_train ,
                method='rf',
                trControl=tc,
                tuneLength = 3,
                importance=TRUE,
                metric='RMSE')
modelo

In [0]:
# Vista de la importancia
#
# finalModel: muestra el tipo de modelo creado, el valor de los hiperparametros 
#    e información adicional, entre ella, el error de entrenamiento
#
varImp(modelo)
modelo$finalModel

In [0]:
# Valores metricas
#
# getTrainPerf: proporciona los resultados de rendimiento promedio de 
#   los mejores parametros ajustados promediados en los pliegues de 
#   validaciones cruzadas repetidas
#
getTrainPerf(modelo)

In [0]:
# Evaluar modelo
#
# predict: Hacer un objeto con predicciones de un objeto modelo ajustado
#     a un entrenamiento previo
# postResample: Dados dos vectores numericos de datos, se calculan el error 
#     cuadratico medio y el R cuadrado
#
set.seed(123)
modelo_pred <- predict(modelo, 
                       newdata = datos_test ) 
postResample(modelo_pred, 
             datos_test$NO2)

In [0]:
####################################
# 2.
# Metodo de entrenamiento oob sobre randomForest
# 
# oob: es un método para medir el error de predicción de randomForest
#
set.seed(123)
tc <- trainControl("oob",
                   number=6, 
                   repeats=2, 
                   verboseIter = FALSE,
                   allowParallel = TRUE)
modelo <- train(NO2~.,
                data=datos_train,
                method='rf',
                trControl=tc,
                tuneLength=3,
                importance=TRUE,
                metric='RMSE')
modelo

In [0]:
# Vista de la importancia
#
varImp(modelo)
modelo$finalModel

In [0]:
# Valores metricas
#
getTrainPerf(modelo)

In [0]:
# Evaluar modelo
#
set.seed(123)
modelo_pred <- predict(modelo, 
                       newdata = datos_test ) 
postResample(modelo_pred, 
             datos_test$NO2)

In [0]:
####################################
# 3.
# Metodo de entrenamiento boot sobre randomForest
# 
# boot: se utiliza para cuantificar la incertidumbre asociada con un 
# estimador dado o un enfoque de aprendizaje estadístico
#
set.seed(123)
tc <- trainControl("boot",
                   number=6, 
                   repeats=2, 
                   verboseIter = FALSE,
                   allowParallel = TRUE)
modelo <- train(NO2~.,
                data=datos_train,
                method='rf',
                trControl=tc,
                tuneLength=3,
                importance=TRUE,
                metric='RMSE')
modelo

In [0]:
# Vista de la importancia
#
varImp(modelo)
modelo$finalModel

In [0]:
# Valores metricas
#
getTrainPerf(modelo)

In [0]:
# Evaluar modelo
#
set.seed(123)
modelo_pred <- predict(modelo, 
                       newdata = datos_test ) 
postResample(modelo_pred, 
             datos_test$NO2)

In [0]:
#####################################
#         SVM_Linear                #
#####################################

##################################
# 1.
# Metodo de entrenamiento boot sobre SVM-Linear
#
# svm-linear: SVM es un modelo que representa a los puntos de muestra 
# en el espacio
#
set.seed(123)
tc <- trainControl("boot",
                   number=5,
                   verboseIter = FALSE,
                   allowParallel = TRUE)
modelo <- train(NO2~.,
                data=datos_train,
                method='svmLinear',
                trControl=tc,
                tuneLength=3,
                importance=TRUE,
                metric='RMSE')
modelo

In [0]:
# Vista de la importancia
#
varImp(modelo)
modelo$finalMode

In [0]:
# Valores metricas
#
getTrainPerf(modelo)

In [0]:
# Evaluar modelo
#
set.seed(123)
modelo_pred <- predict(modelo, 
                       newdata = datos_test ) 
postResample(modelo_pred, 
             datos_test$NO2)

In [0]:
#####################################
#         LM                        #
#####################################

##################################
# 1.
# Metodo de entrenamiento boot sobre LM
#
# lm: regresión lineal
#
set.seed(123)
tc <- trainControl("boot",
                   number=6, 
                   allowParallel = TRUE)
modelo <- train(NO2~.,
                data=datos_train,
                method='lm',
                trControl=tc,
                tuneLength=3,
                metric='RMSE')
modelo

In [0]:
# Vista de la importancia
#
varImp(modelo)
modelo$finalModel

In [0]:
# Valores metricas
#
getTrainPerf(modelo)

In [0]:
# Evaluar modelo
#
set.seed(123)
modelo_pred <- predict(modelo, 
                       newdata = datos_test ) 
postResample(modelo_pred, 
             datos_test$NO2)